In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from scipy.spatial import ConvexHull
import cv2
from scipy.spatial.distance import cdist

### UTILITY FUNCTIONS

In [45]:
WIDTH, HEIGHT = 512, 512
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
MARGIN = 20
YELLOW = (0, 215, 255)
GREEN = (0, 255, 0)

In [3]:
# Reads a particular csv file and returns a list of curves
def read_csv(csv_path):
    np_path_XYs = np.genfromtxt(csv_path, delimiter=',')
    path_XYs = []
    for i in np.unique(np_path_XYs[:, 0]):
        npXYs = np_path_XYs[np_path_XYs[:, 0] == i][:, 1:]
        XYs = []
        for j in np.unique(npXYs[:, 0]):
            XY = npXYs[npXYs[:, 0] == j][:, 1:]
            XYs.append(XY)
            path_XYs.append(XYs[0])
    return path_XYs

# Plots a list of curves
def plot(paths_XYs):
    fig, ax = plt.subplots(tight_layout=True, figsize=(8, 8))
    colours = ['b', 'g', 'r', 'c', 'm', 'y', 'k']
    for i, XYs in enumerate(paths_XYs):
        c = colours[i % len(colours)]
        for XY in XYs:
            ax.plot(XY[:, 0], XY[:, 1], c=c, linewidth=2)
    ax.set_aspect('equal')
    plt.show()

# Reads a csv file and plots the shape detailed there
def plot_shape(csv_path):
    csv_curve = read_csv(csv_path)
    plot(csv_curve)
    print("Number of curves:", len(csv_curve))
    
# Returns the corner points for a set of points
def get_corners(points, epsilon_factor=0.02):
    points = np.array(points, dtype=np.float32).reshape((-1, 1, 2))
    perimeter = cv2.arcLength(points, True)    
    # Approximate the contour with a polygon
    epsilon = epsilon_factor * perimeter
    approx = cv2.approxPolyDP(points, epsilon, True)    
    # Extract corners from the approximation
    corners = [tuple(pt[0]) for pt in approx]
    return corners

def plot_shape_and_corners(points, corners):
    points = np.array(points)
    plt.figure(figsize=(8, 6))
    plt.plot(points[:, 0], points[:, 1], 'o-', label='Original Shape')
    corners = np.array(corners)
    plt.plot(corners[:, 0], corners[:, 1], 'ro', label='Corners')
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.legend()
    plt.title('Shape and Corners')
    plt.show()

def plot_convex_hull(points):
    points = np.array(points)    
    hull = ConvexHull(points)
    hull_points = points[hull.vertices]    
    plt.figure(figsize=(8, 8))
    plt.scatter(points[:, 0], points[:, 1], color='red', s=10, label='Original Points')
    plt.plot(np.append(hull_points[:, 0], hull_points[0, 0]), np.append(hull_points[:, 1], hull_points[0, 1]), color='blue', linestyle='-', label='Convex Hull')
    plt.gca().set_aspect('equal', adjustable='box')
    plt.legend()
    plt.title('Original Points and Convex Hull')
    plt.show()
    
def showCv2Image(img, title = "Image"):
    cv2.imshow(title, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
def checkBounds(x, y):
    if x < 0 or y < 0 :
        return False
    if x >= WIDTH or y >= HEIGHT:
        return False
    return True

### Laying out the foundation

We have multiple filters that will act on the doodle one by one to alter the array of points and the point mask (which indicates whether the subsequent filter should act on a given point or not)

In [46]:
class Filter:
    def __init__(self):
        pass
    def __call__(self, curves, pointMask, image, symmImage, bboxes):
        return curves, pointMask, image, symmImage, bboxes
    
class Mask:
    def __init__(self):
        self.data = dict()
        
    @staticmethod
    def convertKey(key):
        return tuple(list(key))
    
    def __getitem__(self, key):
        return self.data[Mask.convertKey(key)]
    
    def __setitem__(self, key, value):
        self.data[Mask.convertKey(key)] = value
        
    def __delitem__(self, key):
        del self.data[Mask.convertKey(key)]
        
    
class Doodle:
    def computeMx(self):
        allpoints = np.concatenate(self.curves, axis=0)
        self.mn_x = np.min(allpoints[:, 0])
        self.mn_y = np.min(allpoints[:, 1])
        self.mx_x = np.max(allpoints[:, 0])
        self.mx_y = np.max(allpoints[:, 1])
            
    def __init__(self, curves):
        self.curves = curves
        self.image = np.full((512, 512), 255, dtype=np.uint8)
        self.symmImage = np.full((512, 512), 255, dtype=np.uint8)
        self.bboxes = []
        self.filters = []
        self.pointMask = Mask()
        
        self.computeMx()
        scaleFactor = (WIDTH - 2 * MARGIN)/max(self.mx_x, self.mx_y)
        for i, curve in enumerate(self.curves):
            for j, point in enumerate(curve):
                self.curves[i][j] = point * [scaleFactor, scaleFactor] + [MARGIN, MARGIN]
                
        for curve in self.curves:
            for point in curve:
                self.pointMask[point] = 1
        self.computeMx()
        
    def plotCurves(self):
        image = np.full((WIDTH, HEIGHT), 255, np.uint8)
        for curve in self.curves:
            points = np.array(curve).astype(np.int32).reshape(-1, 1, 2)
            cv2.polylines(image, [points], isClosed=False, color = BLACK, thickness=2)
            
        cv2.imshow("Curves", image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        
    def addFilter(self, filter : Filter):
        self.filters.append(filter)
        return self
    
    def build(self):
        for filter in self.filters:
            self.curves, self.pointMask, self.image, self.pointMask, self.bboxes = filter(self.curves, self.pointMask, self.image, self.symmImage, self.bboxes)
        return self

### Filters for regular curves

In [47]:
# This filter straightens all straight lines. It doesn't remove them, just straightens them
class LineFilter(Filter):
    def is_straight_line(self, curve, threshold=0.99):
        x = np.array([p[0] for p in curve]).reshape(-1, 1)
        y = np.array([p[1] for p in curve])
        
        model = LinearRegression().fit(x, y)
        y_pred = model.predict(x)
        
        r2 = r2_score(y, y_pred)
        return r2 >= threshold
    
    def fitline(self, curve):
        return curve
    
    def __call__(self, curves, pointMask, image, symmImage, bboxes):
        newcurves = []
        for curve in curves:
            newcurve = curve
            if self.is_straight_line(curve) : 
                newcurve = self.fitline(curve)
            else :
                newcurves.append(curve)
            
        return newcurves, pointMask, image, symmImage, bboxes

In [53]:
class ConvexFilter(Filter):
    def is_convex_shape(self, curve, threshold=100):
        curve = np.array(curve)
        # Compute the convex hull
        hull = ConvexHull(curve)
        
        # Compute the area of the original shape
        original_area = 0.5 * np.abs(np.dot(curve[:, 0], np.roll(curve[:, 1], 1)) - np.dot(curve[:, 1], np.roll(curve[:, 0], 1)))
        
        # Compute the area of the convex hull
        hull_points = curve[hull.vertices]
        hull_area = 0.5 * np.abs(np.dot(hull_points[:, 0], np.roll(hull_points[:, 1], 1)) - np.dot(hull_points[:, 1], np.roll(hull_points[:, 0], 1)))
        
        # Difference in area
        area_difference = hull_area - original_area
        
        return area_difference <= threshold
    
    
    def is_ellipse_or_circle(self, curve, threshold=0.95):
        curve = np.array(curve)
    
        # Compute area using the Shoelace formula
        def polygon_area(pts):
            return 0.5 * np.abs(np.dot(pts[:, 0], np.roll(pts[:, 1], 1)) - np.dot(pts[:, 1], np.roll(pts[:, 0], 1)))
        
        # Compute perimeter
        def polygon_perimeter(pts):
            return np.sum(np.sqrt(np.sum(np.diff(np.vstack([pts, pts[0]]), axis=0)**2, axis=1)))
        
        area = polygon_area(curve)
        perimeter = polygon_perimeter(curve)
        
        # Compute circularity
        circularity = 4 * np.pi * area / (perimeter ** 2)
    
        return circularity >= threshold
    
    
    def is_circle(self, curve, threshold=0.8):
        # get bounding box around shape
        curve = np.array(curve)
        min_x = np.min(curve[:, 0])
        max_x = np.max(curve[:, 0])
        min_y = np.min(curve[:, 1])
        max_y = np.max(curve[:, 1])

        # find aspect ratio of bounding box
        width = max_x - min_x
        height = max_y - min_y
        if height < width :
            width, height = height, width
        aspect_ratio = width / height

        return aspect_ratio >= threshold
    

    def fit_circle(self, curve, image, symmImage):
        def get_circle(circle_list):
            # calculate circle center using mean
            x, y = zip(*circle_list)
            x_center = np.mean(x)
            y_center = np.mean(y)

            # rearrange points such that origin becomes center
            u = x - x_center
            v = y - y_center

            # get radius by solving least mean squared error 
            distances = np.sqrt(u**2 + v**2)
            radius = np.mean(distances)

            # print(x_center, y_center, radius)
            return x_center, y_center, radius
        
        x_center, y_center, radius = get_circle(curve)

        # Convert center coordinates to integers
        x_center = int(x_center)
        y_center = int(y_center)
        radius = int(radius)

        cv2.circle(image, (x_center, y_center), radius, BLACK, 2)
        cv2.circle(symmImage, (x_center, y_center), radius, BLACK, 2)

        # Mark the center of the circle (radial symmetry point)
        cv2.drawMarker(symmImage, (x_center, y_center), color=(255, 0, 0), markerType=cv2.MARKER_CROSS, markerSize=20, thickness=2)
        
        return image, symmImage
    

    def fit_ellipse(self, curve, image, symmImage):
        # Convert the list of points to a numpy array
        curve = np.array(curve, dtype=np.float32)
        
        # Fit an ellipse to the points
        ellipse = cv2.fitEllipse(curve)
        
        # Extract ellipse parameters
        center, axes, angle = ellipse
        major_axis, minor_axis = axes
        
        # Create points along the ellipse
        theta = np.linspace(0, 2 * np.pi, 100)
        ellipse_points = np.array([
            center[0] + major_axis / 2 * np.cos(theta) * np.cos(np.radians(angle)) - minor_axis / 2 * np.sin(theta) * np.sin(np.radians(angle)),
            center[1] + major_axis / 2 * np.cos(theta) * np.sin(np.radians(angle)) + minor_axis / 2 * np.sin(theta) * np.cos(np.radians(angle))
        ]).T

        cv2.polylines(image, [ellipse_points.reshape(-1, 1, 2)], isClosed=True, color = BLACK, thickness=2)
        cv2.polylines(symmImage, [ellipse_points.reshape(-1, 1, 2)], isClosed=True, color = BLACK, thickness=2)

        # Major and minor axis unit vectors
        major_axis_vector = np.array([
            np.cos(np.radians(angle)),
            np.sin(np.radians(angle))
        ])
        minor_axis_vector = np.array([
            -np.sin(np.radians(angle)),
            np.cos(np.radians(angle))
        ])

        # Extend the major and minor axes to the edges of the ellipse
        major_axis_start = center - (major_axis / 2) * major_axis_vector
        major_axis_end = center + (major_axis / 2) * major_axis_vector
        
        minor_axis_start = center - (minor_axis / 2) * minor_axis_vector
        minor_axis_end = center + (minor_axis / 2) * minor_axis_vector

        # Convert the start and end points of the major and minor axes to integers
        major_axis_start = tuple(map(int, major_axis_start))
        major_axis_end = tuple(map(int, major_axis_end))

        minor_axis_start = tuple(map(int, minor_axis_start))
        minor_axis_end = tuple(map(int, minor_axis_end))

        # Draw the major axis (extended to the edges of the ellipse)
        cv2.line(symmImage, major_axis_start, major_axis_end, color=YELLOW, thickness=2)

        # Draw the minor axis (extended to the edges of the ellipse)
        cv2.line(symmImage, minor_axis_start, minor_axis_end, color=YELLOW, thickness=2)

        return image, symmImage
    

    def fit_rectangle(self, curve, image, symmImage):
        # Convert the list of points to a numpy array
        curve = np.array(curve, dtype=np.float32)
        
        # Fit a rotated rectangle to the points
        rect = cv2.minAreaRect(curve)
        box = cv2.boxPoints(rect)
        box = np.int0(box)
        
        top_left = tuple(box[0])
        bottom_right = tuple(box[2])

        cv2.rectangle(image, top_left, bottom_right, color=BLACK, thickness=2)
        cv2.rectangle(symmImage, top_left, bottom_right, color=BLACK, thickness=2)

        # Draw lines of symmetry with a small extension beyond the rectangle
        extension_factor = 0.2  # Extend the lines by 20% of the rectangle's width or height

        # Calculate midpoints of the sides
        midpoints = [(box[i] + box[(i + 1) % 4]) / 2 for i in range(4)]

        # Convert midpoints to integer tuples
        midpoints = [tuple(map(int, midpoint)) for midpoint in midpoints]

        # Calculate the direction vectors for the symmetry lines
        direction_vectors = [midpoints[i + 2] - midpoints[i] for i in range(2)]

        # Draw the symmetry lines with an extension
        for i in range(2):
            # Calculate the extended start and end points
            extended_start = midpoints[i] - extension_factor * direction_vectors[i]
            extended_end = midpoints[i + 2] + extension_factor * direction_vectors[i]
            
            # Convert the extended points to integers
            extended_start = tuple(map(int, extended_start))
            extended_end = tuple(map(int, extended_end))
            
            # Draw the line
            cv2.line(symmImage, extended_start, extended_end, color=YELLOW, thickness=2)

        return image, symmImage
    

    def fit_polygon(self, curve, image, symmImage):
        # Get corners of the approximated polygon
        corners = get_corners(curve)
        points = np.array(curve)
        corners = np.array(corners)

        # Calculate centroid of corners
        centroid = np.mean(corners, axis=0)

        # Find the point with the highest y-value to fix its position
        fixed_point = corners[np.argmax(corners[:, 1])]
        
        # Calculate radius from centroid to the fixed point
        radius = np.linalg.norm(fixed_point - centroid)
        
        # Calculate initial angle of the fixed point from the centroid
        initial_angle = np.arctan2(fixed_point[1] - centroid[1], fixed_point[0] - centroid[0])
        
        # Number of vertices
        num_vertices = len(corners)
        
        # Generate the regular polygon
        regular_polygon = []
        
            
        for i in range(num_vertices):
            angle = initial_angle + 2 * np.pi * i / num_vertices
            x = centroid[0] + radius * np.cos(angle)
            y = centroid[1] + radius * np.sin(angle)
            if not checkBounds(x, y):
                continue
            regular_polygon.append((x, y))
        
        regular_polygon = np.array(regular_polygon).astype(np.int32).reshape(-1, 1, 2)
        cv2.polylines(image, [regular_polygon], isClosed=True, color = BLACK, thickness=2)
        cv2.polylines(symmImage, [regular_polygon], isClosed=True, color = BLACK, thickness=2)

        # Draw lines of symmetry with a small extension beyond the shape
        extension_factor = 0.2  # Extend the lines by 20% of the distance from the centroid to the corners

        # Convert points to integer tuples for OpenCV
        regular_polygon = [tuple(map(int, pt)) for pt in np.array(regular_polygon).reshape(-1, 2)]
        centroid = tuple(map(int, np.array(centroid).flatten()))

        # Draw lines of symmetry
        if num_vertices % 2 == 0:
            # Even number of vertices
            for i in range(num_vertices):
                # Case 1: Line from vertex to opposite vertex
                start_point = regular_polygon[i]
                end_point = regular_polygon[(i + num_vertices // 2) % num_vertices]
                direction_vector = np.array(end_point) - np.array(start_point)
                extended_start = tuple(map(int, np.array(start_point) - extension_factor * direction_vector))
                extended_end = tuple(map(int, np.array(end_point) + extension_factor * direction_vector))
                cv2.line(symmImage, extended_start, extended_end, color=YELLOW, thickness=2)

                # Case 2: Line from midpoint of one side to midpoint of opposite side
                start_mid_point = tuple(map(int, (np.array(regular_polygon[i]) + np.array(regular_polygon[(i + 1) % num_vertices])) / 2))
                end_mid_point = tuple(map(int, (np.array(regular_polygon[(i + num_vertices // 2) % num_vertices]) + 
                                                np.array(regular_polygon[(i + num_vertices // 2 + 1) % num_vertices])) / 2))
                direction_vector = np.array(end_mid_point) - np.array(start_mid_point)
                extended_start_mid = tuple(map(int, np.array(start_mid_point) - extension_factor * direction_vector))
                extended_end_mid = tuple(map(int, np.array(end_mid_point) + extension_factor * direction_vector))
                cv2.line(symmImage, extended_start_mid, extended_end_mid, color=YELLOW, thickness=2)
        else:
            # Odd number of vertices
            for i in range(num_vertices):
                start_point = regular_polygon[i]
                opposite_index = (i + num_vertices // 2) % num_vertices
                opposite_side_midpoint = tuple(map(int, (np.array(regular_polygon[opposite_index]) + 
                                                        np.array(regular_polygon[(opposite_index + 1) % num_vertices])) / 2))
                direction_vector = np.array(opposite_side_midpoint) - np.array(start_point)
                extended_start = tuple(map(int, np.array(start_point) - extension_factor * direction_vector))
                extended_end = tuple(map(int, np.array(opposite_side_midpoint) + extension_factor * direction_vector))
                cv2.line(symmImage, extended_start, extended_end, color=YELLOW, thickness=2)

        # Draw dotted radial symmetry lines from centroid to slightly beyond corners
        for corner in regular_polygon:
            direction_vector = np.array(corner) - np.array(centroid)
            extended_point = tuple(map(int, np.array(corner) + 0.5 * direction_vector))
            # Draw dotted line manually
            line_length = np.linalg.norm(np.array(extended_point) - np.array(centroid))
            num_dots = int(line_length // 10)  # Control the spacing of dots
            for j in range(num_dots):
                start_dot = tuple(map(int, np.array(centroid) + j / num_dots * (np.array(extended_point) - np.array(centroid))))
                end_dot = tuple(map(int, np.array(centroid) + (j + 0.5) / num_dots * (np.array(extended_point) - np.array(centroid))))
                cv2.line(symmImage, start_dot, end_dot, color=GREEN, thickness=2)

        return image, symmImage
    

    def __call__(self, curves, pointMask, image, symmImage, bboxes):
        newcurves = []
        for curve in curves:
            if self.is_convex_shape(curve):
                if self.is_ellipse_or_circle(curve):
                    if self.is_circle(curve):
                        image, symmImage = self.fit_circle(curve, image, symmImage) 
                    else :
                        image, symmImage = self.fit_ellipse(curve, image, symmImage)
                else :
                    # newcurves.append(curve)
                    image, symmImage = self.fit_polygon(curve, image, symmImage)
            else :
                newcurves.append(curve)
                
        return newcurves, pointMask, image, symmImage, bboxes

In [54]:
doodle = Doodle(read_csv(r"C:\Users\suyash\OneDrive - iiit-b\Projects\GenSolve_Git\Curvetopia1\problems\problems\isolated.csv"))
doodle.plotCurves()
doodle = doodle.addFilter(LineFilter()).addFilter(ConvexFilter()).build()

In [55]:
doodle.plotCurves()

In [56]:
showCv2Image(doodle.image)

In [58]:
showCv2Image(doodle.symmImage)